<a href="https://colab.research.google.com/github/MauroGonzalez51/WaveAnalysis-in-React/blob/master/WaveAnalisis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# * In case of installing new libraries use pip install <!- LIBRARY ->

In [2]:
pip install numpy

In [3]:
pip install librosa

In [4]:
pip install flask_ngrok

In [5]:
pip install flask_cors

In [6]:
pip install pydub

In [7]:
from IPython.core.display import json

In [8]:
import numpy as np
import librosa
from google.colab import files
import librosa.display
import matplotlib.pyplot as plt
import os
from flask import Flask, request, jsonify, send_file
from flask_cors import CORS
from flask_ngrok import run_with_ngrok
import io
import zipfile
from pydub import AudioSegment

In [9]:
# ! Select a file

def select_file():
  return files.upload()

In [10]:
def get_wave_shape(y, sr, save = False):
    plt.figure(figsize=(12,4))
    librosa.display.waveshow(y, sr=sr)
    plt.title("Forma de Onda")
    plt.xlabel("Tiempo (s)")
    plt.ylabel("Amplitud")

    if (save):
        # * Save the file in a BytesIO Object
        img_buffer = io.BytesIO()
        plt.savefig(img_buffer, format='png')
        img_buffer.seek(0)
        return img_buffer

    else:
        plt.show()
        plt.close()
        return None

def get_spectogram(y, sr, save=False):
    S = np.abs(librosa.stft(y))
    D = librosa.amplitude_to_db(S, ref=np.max)
    plt.figure(figsize=(12, 4))
    librosa.display.specshow(D, sr=sr, x_axis="time", y_axis="log")
    plt.colorbar(format="%+2.0f dB")
    plt.title("Espectrograma")

    if save:
        # Save the file in a BytesIO Object
        img_buffer = io.BytesIO()
        plt.savefig(img_buffer, format='png')
        img_buffer.seek(0)
        return img_buffer
    else:
        plt.show()
        plt.close()
        return None

def get_tempo_rhythm(y, sr, save = False):
    tempo, beat_frames = librosa.beat.beat_track(y=y, sr=sr)

    if (save):
        return {
            "tempo": tempo
        }

    else:
        print(f"Tempo estimado: ${tempo} BMP")

def get_tonality_features(y, sr, save = False):
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    plt.figure(figsize=(12, 4))
    librosa.display.specshow(chroma, y_axis="chroma", x_axis="time")
    plt.colorbar()
    plt.title("Características de Tonalidad")

    if (save):
        # * Save the file in a BytesIO Object
        img_buffer = io.BytesIO()
        plt.savefig(img_buffer, format='png')
        img_buffer.seek(0)
        return img_buffer

    else:
        plt.show()
        plt.close()
        return None

def calculate_mfcc(y, sr, save = False):
    mfccs = librosa.feature.mfcc(y=y, sr=sr)
    plt.figure(figsize=(12, 4))
    librosa.display.specshow(mfccs, x_axis="time")
    plt.colorbar()
    plt.title("MFCC")

    if (save):
        # * Save the file in a BytesIO Object
        img_buffer = io.BytesIO()
        plt.savefig(img_buffer, format='png')
        img_buffer.seek(0)
        return img_buffer

    else:
        plt.show()
        plt.close()
        return None

In [11]:
def analize_audio(selected_file, save=False):
    try:
        # Load the audio file
        y, sr = librosa.load(selected_file)

        images = []

        if not save:
            # Visualize the shape of the wave
            get_wave_shape(y, sr, save)

            # Extract the spectrogram
            get_spectogram(y, sr, save)

            # Tempo and rhythm
            get_tempo_rhythm(y, sr, save)

            # Extract tonality features
            get_tonality_features(y, sr, save)

            # Calculate the MFCC (Mel-frequency cepstral coefficients)
            calculate_mfcc(y, sr, save)

        else:
            # Generate and save images
            images.append(("wave_shape.png", get_wave_shape(y, sr, save)))
            images.append(("spectogram.png", get_spectogram(y, sr, save)))
            images.append(("tonality_features.png", get_tonality_features(y, sr, save)))
            images.append(("mfcc.png", calculate_mfcc(y, sr, save)))

            # Create a ZIP file
            zip_buffer = io.BytesIO()
            with zipfile.ZipFile(zip_buffer, "a") as zipf:
                for img_filename, img_buffer in images:
                    if img_buffer:
                        zipf.writestr(img_filename, img_buffer.getvalue())

            tempo_rhythm = get_tempo_rhythm(y, sr, save)
            with zipfile.ZipFile(zip_buffer, "a") as zipf:
                zipf.writestr("tempo_rhythm.json", json.dumps(tempo_rhythm))

            zip_buffer.seek(0)
            return zip_buffer

    except Exception as e:
        print(f"Exception: {e}")
        return None


In [12]:
# def main():
#   uploaded_files = select_file()

#   for filename, file_content in uploaded_files.items():
#       filename = "/content/" + "file.mp3"

#       # Save the content in a file with the new name
#       with open(filename, "wb") as f:
#           f.write(file_content)

#       # Finally, analysing the audio
#       analize_audio(filename, False)

# main()

In [ ]:
app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

@app.route("/", methods=["GET"])
def connection_stablished():
    return jsonify({ "message": "Hello there!" })

@app.route('/analyze', methods=["POST"])
def analyze():
    try:
        if 'audio' in request.files:
            audio = request.files['audio']
            zip_file = analize_audio(audio, True)
            return send_file(zip_file, mimetype='application/zip', as_attachment=True, download_name='analysis.zip')
        else:
            return jsonify({ "message": "File not found in request!" })
    except Exception as e:
        print(f"Exception: {e}")
        return jsonify({ "error": str(e) })


app.config['MAX_CONTENT_LENGTH'] = 30 * 1024 * 1024  # Max content size: 30 MB

app.run()